# Packages

In [2]:
# Use autoreload to automatically reload modules
%load_ext autoreload
%autoreload 2

In [3]:
# Setup paths
import rootutils
import xarray as xr
import pandas as pd
root_path = rootutils.find_root()

In [9]:
def img_interactive(da):
    import ipywidgets as widgets
    import matplotlib.pyplot as plt

    def plot_time_steps(time_index1, time_index2):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # Left image
        da.isel(time=time_index1).plot.imshow(robust=True, ax=ax1)
        ax1.set_title(f"{str(da.time.values[time_index1])[:10]}")
        ax1.axis('off')
        
        # Right image  
        da.isel(time=time_index2).plot.imshow(robust=True, ax=ax2)
        ax2.set_title(f"{str(da.time.values[time_index2])[:10]}")
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()

    widgets.interact(
        plot_time_steps, 
        time_index1=widgets.IntSlider(min=0, max=len(da.time)-1, step=1, value=0, description='Left Image'),
        time_index2=widgets.IntSlider(min=0, max=len(da.time)-1, step=1, value=1, description='Right Image')
    )

In [5]:
deforest_df = pd.read_parquet(root_path / "data" / "processed" / "radd" / "south_america_combined_clean.parquet")
deforest_df

,geometry,tile_name,sample_id,component_id,size_pixels,area_hectares,earliest_alert,latest_alert,duration_days,centroid_x,centroid_y,confidence,date_raster_file
0,b'\x01\x01\x00\x00\x00\x03\x16k;l\xbeH\xc0QXZ\...,00N_050W_radd_alerts,sample_07,32,109,1.342989,2021-08-29,2021-09-22,24,-49.487678,-8.150868,high,sample_07_dates.tif
1,b'\x01\x01\x00\x00\x00zw\xf0\xf05\xcdH\xc0\r\x...,00N_050W_radd_alerts,sample_07,111,312,3.844152,2025-05-16,2025-06-09,24,-49.603209,-8.475450,high,sample_07_dates.tif
2,b'\x01\x01\x00\x00\x00\x9a\xb2\xaf\x7f\xd3PF\x...,00N_050W_radd_alerts,sample_11,33,158,1.946718,2024-10-14,2024-11-07,24,-44.631454,-5.697680,high,sample_11_dates.tif
3,b'\x01\x01\x00\x00\x00\x91\xee\x8f@\x06kF\xc0\...,00N_050W_radd_alerts,sample_11,145,106,1.306026,2022-11-06,2022-11-18,12,-44.836128,-5.717239,high,sample_11_dates.tif
4,b'\x01\x01\x00\x00\x00\xd4\x18\xdb\x13NMF\xc0\...,00N_050W_radd_alerts,sample_11,194,126,1.552446,2021-09-19,2021-09-19,0,-44.603945,-5.726981,high,sample_11_dates.tif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,b'\x01\x01\x00\x00\x00\xc9\xed\x84\x84\x15\xed...,20S_060W_radd_alerts,sample_26,535,120,1.478520,2021-09-26,2021-10-20,24,-57.852219,-22.696022,high,sample_26_dates.tif
1584,b'\x01\x01\x00\x00\x00\xdcD\x82m\xa8\xfbL\xc0\...,20S_060W_radd_alerts,sample_26,612,329,4.053609,2021-09-14,2021-10-08,24,-57.966078,-22.769136,high,sample_26_dates.tif
1585,b'\x01\x01\x00\x00\x00\xaa\xfc\xca\xb7$\xfcL\x...,20S_060W_radd_alerts,sample_26,615,129,1.589409,2021-09-14,2021-10-08,24,-57.969871,-22.770695,high,sample_26_dates.tif
1586,b'\x01\x01\x00\x00\x00\x98\x86\xe1#b\xfcL\xc0c...,20S_060W_radd_alerts,sample_26,617,220,2.710620,2021-09-14,2021-10-08,24,-57.971745,-22.772191,high,sample_26_dates.tif


In [13]:

i = 34
print(f"Earliest alert: {deforest_df['earliest_alert'].iloc[i]}")
print(f"Latest alert: {deforest_df['latest_alert'].iloc[i]}")
da_s2 = xr.open_dataarray(root_path / "data" / "processed" / "sen_data" / f"event_{i}" /"da_s2.nc" ) 
da_s2_rgb = da_s2.sel(band=["B04", "B03", "B02"])
img_interactive(da_s2_rgb)

Earliest alert: 2024-11-17
Latest alert: 2024-11-29


interactive(children=(IntSlider(value=0, description='Left Image', max=13), IntSlider(value=1, description='Ri…